In [27]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import glm
import collections
from statsmodels.sandbox.stats.multicomp import multipletests

In [28]:
df=pd.read_csv('Final_poisson.csv').iloc[:,1:]

In [38]:
df.columns

Index(['FECHA_ACCIDENTE', 'AÑO_ACCIDENTE', 'MES_ACCIDENTE', 'DIA_ACCIDENTE',
       'HORA_ACCIDENTE', 'GRAVEDAD_ACCIDENTE', 'CLASE_ACCIDENTE',
       'SITIO_EXACTO_ACCIDENTE', 'CANT_HERIDOS_EN _SITIO_ACCIDENTE',
       'CANT_MUERTOS_EN _SITIO_ACCIDENTE', 'LONGITUD', 'LATITUD', 'POT',
       'PIEZA_URBANA', 'BUS_SUBIDA', 'BUS_BAJADA', 'TRAFICO', 'ES_FESTIVO',
       'TIPO_FESTIVO', 'DIA_DEL_AÑO', 'SEMANA_DEL_AÑO', 'CUATRIMESTRE_DEL_AÑO',
       'JUNIOR_LOCAL', 'JUNIOR_VISITANTE', 'RESULTADO_LOCAL',
       'RESULTADO_VISITANTE', 'JUNIOR_GANO', 'RELEVANCIA_PARTIDO',
       'HIPOTESIS_ACCIDENTE', 'VEHICULO_MOTOCICLETA', 'VEHICULO_AUTOMOVIL',
       'VEHICULO_CAMIONETA', 'VEHICULO_BUS', 'VEHICULO_CAMION',
       'VEHICULO_CAMPERO', 'VEHICULO_BUSETA', 'VEHICULO_BICICLETA',
       'VEHICULO_MICROBUS', 'VEHICULO_TRACTO_CAMION', 'VEHICULO_VOLQUETA',
       'VEHICULO_MOTOCARRO', 'VEHICULO_DESCONOCIDO', 'VEHICULO_CUATRIMOTO',
       'VEHICULO_MAQUINARIA_AGRICOLA', 'VEHICULO_TRACCION_ANIMAL',
    

In [30]:
import time

In [31]:
def conteo_perf(newdf=df):
    df_drop=newdf.drop_duplicates(subset='LAT_LON') 
    conteos=newdf[['LAT_LON','SITIO_EXACTO_ACCIDENTE']].groupby(['LAT_LON','SITIO_EXACTO_ACCIDENTE']).size().reset_index(name='count')
    grp_conteos = conteos.groupby('LAT_LON')['count'].agg(['sum','max'])
    grp_conteos['sum_rate'] = grp_conteos['max'] / grp_conteos['sum']
    filtered_conteos = grp_conteos[grp_conteos['sum_rate'] > 0.3]
    return_df = filtered_conteos.drop(['max', 'sum_rate'], axis=1)
    return_df.rename(columns = {'sum':'NACCIDENTS'}, inplace = True) 
    fusion=pd.merge(return_df,df_drop,on='LAT_LON',how='inner')[['NACCIDENTS','AÑO_ACCIDENTE','MES_ACCIDENTE',
                                                     'CLASE_ACCIDENTE','LONGITUD', 'LATITUD',
                                                     'POT', 'PIEZA_URBANA', 'BUS_SUBIDA','BUS_BAJADA', 
                                                     'TRAFICO','ES_FESTIVO','SOLO_HORA','DIA_ACCIDENTE']].reset_index().iloc[:,1:]
    return(fusion)

start_time = time.time()
b=conteo_perf()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.15281128883361816 seconds ---


In [32]:
semaforos=pd.read_csv('lights_clean.csv')[['LATITUD','LONGITUD']]

In [71]:
def distancia_semaforos_mas_rapido(final_df=b,lights=semaforos):
    closest=np.zeros(final_df.shape[0])
    numclose=np.zeros(final_df.shape[0])
    numclose2=np.zeros(final_df.shape[0])
    numclose3=np.zeros(final_df.shape[0])
    for i in range(final_df.shape[0]):
        corner_lat=final_df['LATITUD'][i]
        corner_lon=final_df['LONGITUD'][i]
        light_lat=np.array(lights['LATITUD'])
        light_lon=np.array(lights['LONGITUD'])
        dx = (light_lon-corner_lon)*40000*np.cos((light_lat+corner_lat)*math.pi/360)/360
        dy = (light_lat-corner_lat)*40000/360
        distance=np.sqrt(dx**2+dy**2)
        closest[i]=distance[distance.argmin()]
        numclose[i]=np.sum(distance<0.1)
        numclose2[i]=np.sum(((distance>=0.1) & (distance<0.5)))
        numclose3[i]=np.sum(((distance>=0.5) & (distance<2)))
    final_df['NLIGHTS']=list(numclose)
    final_df['NLIGHTS2']=list(numclose2)
    final_df['NLIGHTS3']=list(numclose3)
    final_df['CLOSEST_LIGHT']=list(closest)
    return(final_df)

In [34]:
start_time = time.time()
distancia_semaforos_mas_rapido()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.9632720947265625 seconds ---


In [70]:
df.DIA_ACCIDENTE

0        Mié
1        Mié
2        Mié
3        Mié
4        Mié
        ... 
21350    Vie
21351    Vie
21352    Vie
21353    Vie
21354    Vie
Name: DIA_ACCIDENTE, Length: 21355, dtype: object

In [36]:
def modelo_poisson(dia=False,festivo=False,mes=False,año=True df=df,lights=semaforos): #Ojo dia, festivo, mes, año tienen que ser excluyente, si uno es verdadero automaticamente los otros son falsos
        if(año==True):
            df=df
        elif(mes!=False):
            df=df[df.MES_ACCIDENTE==mes]
        elif(festivo!=False):
            df=df[df.ES_FESTIVO==festivo]
        elif(dia!=False):
            df=df[df.DIA_ACCIDENTE==dia]
        data=conteo_perf(newdf=df)
        data=distancia_semaforos_mas_rapido(final_df=data,lights=lights)
        formula='NACCIDENTS~BUS_SUBIDA+BUS_BAJADA+POT+PIEZA_URBANA+NLIGHTS+NLIGHTS2+TRAFICO+CLOSEST_LIGHT'
        model = glm(formula=formula, data=data, family=sm.families.Poisson()).fit()
        predichos=model.predict()
        data=
        
        


SyntaxError: invalid syntax (<ipython-input-36-a6589c3753ff>, line 1)